# Catchment Scale Data Analysis

## 1. Preparing the data

In [1]:
## Importing library and setting up
library(stringr)
library(ggplot2)
library(dplyr)

rm(list=ls())


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
#change this to working directory
stew_name = "Strockhorsterdijk"
dir = paste("D:/Wageningen/Period 1/EDCA/Part 2/Stew", stew_name)
setwd(dir)

In [3]:
discharge = read.csv('Discharge.csv', skip=16, sep=";")
upstream = read.csv('Upstream.csv', skip=16, sep=";")
downstream = read.csv('Downstream.csv', skip=16, sep=";")
valve = read.csv('Valve.csv', skip=16, sep=";")

In [4]:
#rename columns
colnames(discharge)[1]="Date"
colnames(discharge)[2]="Value"

colnames(upstream)[1]="Date"
colnames(upstream)[2]="Value"

colnames(downstream)[1]="Date"
colnames(downstream)[2]="Value"

colnames(valve)[1]="Date"
colnames(valve)[2]="Value"

print(head(discharge))

                 Date Value
1 2010-08-22 15:00:00   ---
2 2010-08-22 16:00:00   ---
3 2010-08-22 17:00:00   ---
4 2010-08-22 18:00:00   ---
5 2010-08-22 19:00:00   ---
6 2010-08-22 20:00:00   ---


In [5]:
#filtering data
discharge$Value=str_trim(discharge$Value)
discharge$Value[discharge$Value == "---"] <- NA
discharge$Value = as.numeric(gsub(",", ".", gsub("\\.", "", discharge$Value)))

upstream$Value=str_trim(upstream$Value)
upstream$Value[upstream$Value == "---"] <- NA
upstream$Value = as.numeric(gsub(",", ".", gsub("\\.", "", upstream$Value)))

downstream$Value=str_trim(downstream$Value)
downstream$Value[downstream$Value == "---"] <- NA
downstream$Value = as.numeric(gsub(",", ".", gsub("\\.", "", downstream$Value)))

valve$Value=str_trim(valve$Value)
valve$Value[valve$Value == "---"] <- NA
valve$Value = as.numeric(gsub(",", ".", gsub("\\.", "", valve$Value)))

print(head(discharge))

                 Date Value
1 2010-08-22 15:00:00    NA
2 2010-08-22 16:00:00    NA
3 2010-08-22 17:00:00    NA
4 2010-08-22 18:00:00    NA
5 2010-08-22 19:00:00    NA
6 2010-08-22 20:00:00    NA


In [6]:
#changing date format to POSIXlt objects
discharge$Date <- as.POSIXlt(discharge$Date, format="%Y-%m-%d %H:%M:%S")
valve$Date <- as.POSIXlt(valve$Date, format="%Y-%m-%d %H:%M:%S")
upstream$Date <- as.POSIXlt(upstream$Date, format="%Y-%m-%d %H:%M:%S")
downstream$Date <- as.POSIXlt(downstream$Date, format="%Y-%m-%d %H:%M:%S")

In [7]:
#start and end full data
start_full <- min(discharge$Date[1],upstream$Date[1],valve$Date[1],downstream$Date[1])
end_full <- max(discharge$Date[length(discharge$Date)],upstream$Date[length(upstream$Date)],valve$Date[length(valve$Date)],downstream$Date[length(downstream$Date)])

#make full data
date_sequence <- seq(from = start_full, to = end_full, by = "hour")  
full_data <- data.frame(Date = date_sequence)

full_data <- left_join(full_data, discharge, by = "Date")
colnames(full_data)[2]="Discharge"

full_data <- left_join(full_data, downstream, by = "Date")
colnames(full_data)[3]="Downstream"

full_data <- left_join(full_data, upstream, by = "Date")
colnames(full_data)[4]="Upstream"

full_data <- left_join(full_data, valve, by = "Date")
colnames(full_data)[5]="Valve"

print(head(full_data))

summary(full_data)

                 Date Discharge Downstream Upstream Valve
1 2010-08-22 15:00:00        NA     17.035   17.654    NA
2 2010-08-22 16:00:00        NA     17.034   17.654    NA
3 2010-08-22 17:00:00        NA     17.034   17.654    NA
4 2010-08-22 18:00:00        NA     17.035   17.654    NA
5 2010-08-22 19:00:00        NA     17.035   17.655    NA
6 2010-08-22 20:00:00        NA     17.036   17.655    NA


      Date                       Discharge       Downstream       Upstream    
 Min.   :2010-08-22 15:00:00   Min.   :0.000   Min.   :16.77   Min.   :17.10  
 1st Qu.:2013-12-02 13:45:00   1st Qu.:0.001   1st Qu.:17.04   1st Qu.:17.72  
 Median :2017-03-14 13:30:00   Median :0.045   Median :17.18   Median :17.77  
 Mean   :2017-03-14 13:30:00   Mean   :0.096   Mean   :17.19   Mean   :17.72  
 3rd Qu.:2020-06-24 14:15:00   3rd Qu.:0.127   3rd Qu.:17.32   3rd Qu.:17.83  
 Max.   :2023-10-05 14:00:00   Max.   :2.089   Max.   :18.59   Max.   :18.61  
                               NA's   :24408   NA's   :23905   NA's   :23947  
     Valve      
 Min.   :17.69  
 1st Qu.:17.70  
 Median :17.73  
 Mean   :17.74  
 3rd Qu.:17.75  
 Max.   :17.96  
 NA's   :18926  

## 2. Filling the Data Using Q-h Relationship

In [8]:
#ommiting NA values and choosing only positive or 0 data
positive_full_data <- full_data[full_data$Upstream-full_data$Valve >= 0,]
positive_full_data <- na.omit(positive_full_data)

print(head(positive_full_data))

                     Date Discharge Downstream Upstream  Valve
12673 2012-02-01 14:00:00     0.070     17.268   17.789 17.735
12674 2012-02-01 15:00:00     0.071     17.267   17.790 17.736
12675 2012-02-01 16:00:00     0.071     17.267   17.790 17.736
12676 2012-02-01 17:00:00     0.069     17.268   17.789 17.736
12677 2012-02-01 18:00:00     0.068     17.268   17.788 17.735
12678 2012-02-01 19:00:00     0.067     17.268   17.786 17.734


In [9]:
#adding h (water level) column
options(repr.plot.width = 13)

positive_full_data$h = positive_full_data$Upstream-positive_full_data$Valve

print(head(positive_full_data))

                     Date Discharge Downstream Upstream  Valve     h
12673 2012-02-01 14:00:00     0.070     17.268   17.789 17.735 0.054
12674 2012-02-01 15:00:00     0.071     17.267   17.790 17.736 0.054
12675 2012-02-01 16:00:00     0.071     17.267   17.790 17.736 0.054
12676 2012-02-01 17:00:00     0.069     17.268   17.789 17.736 0.053
12677 2012-02-01 18:00:00     0.068     17.268   17.788 17.735 0.053
12678 2012-02-01 19:00:00     0.067     17.268   17.786 17.734 0.052


In [10]:
# Specify the file path where you want to save the CSV file
file_path <- paste("D:/Wageningen/Period 1/EDCA/Part 2/Stew ",stew_name,"/output.csv", sep="")

# Export the data frame to a CSV file
write.table(positive_full_data, file = file_path, row.names = FALSE, sep=";")

In [ ]:
lm_model <- lm(positive_full_data$Discharge ~ I(positive_full_data$h^(3/2))-1)

predicted_values <- predict(lm_model)

ggplot(positive_full_data, aes(x = h, y = Discharge)) +
  geom_point(shape = 4, size =3, color = "blue") +
  geom_line(aes(y = predicted_values), color = "red") +
  labs(x = "Height (m)", y = expression(paste("Discharge (", m^3,"/h)")))+
  theme(
      panel.background = element_rect(fill = "lightblue"),
      plot.title = element_text(hjust = 0.5, size=16, face ="bold"),
      axis.title.x = element_text(size = 15),               
      axis.title.y = element_text(size = 15),               
      axis.text.x = element_text(size = 12),                
      axis.text.y = element_text(size = 12)  
  ) +
  ggtitle(paste("Q-h Relationship", stew_name))

In [38]:
?ggplot

In [ ]:
#start and end period
start = "2015-01-01"
end = "2018-01-01"

In [ ]:
#plotting
options(repr.plot.width = 15)

discharge_subset <- discharge[discharge$Date >= as.POSIXlt(start) & discharge$Date <= as.POSIXlt(end), ]
downstream_subset <- downstream[downstream$Date >= as.POSIXlt(start) & downstream$Date <= as.POSIXlt(end), ]
upstream_subset <- upstream[upstream$Date >= as.POSIXlt(start) & upstream$Date <= as.POSIXlt(end), ]
valve_subset <- valve[valve$Date >= as.POSIXlt(start) & valve$Date <= as.POSIXlt(end), ]

#plotting ggplot
ggplot(discharge_subset, aes(x = as.Date(Date), y = Value)) +
  geom_line() +
  labs(y = "Discharge", x = "Date") +
  scale_x_date(date_labels = "%b %Y", date_breaks = "6 months")

ggplot(downstream_subset, aes(x = as.Date(Date), y = Value)) +
  geom_line() +
  labs(y = "Downstream", x = "Date") +
  scale_x_date(date_labels = "%b %Y", date_breaks = "6 months")

ggplot(upstream_subset, aes(x = as.Date(Date), y = Value)) +
  geom_line() +
  labs(y = "Upstream", x = "Date") +
  scale_x_date(date_labels = "%b %Y", date_breaks = "6 months")

ggplot(valve_subset, aes(x = as.Date(Date), y = Value)) +
  geom_line() +
  labs(y = "Valve", x = "Date") +
  scale_x_date(date_labels = "%b %Y", date_breaks = "6 months")


In [ ]:
boxplot(full_data$Discharge,full_data$Downstream)